In [1]:
# Ignore this cell when running on colab
!nvidia-smi

Thu Apr 20 13:07:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          Off  | 00000000:04:00.0 Off |                    0 |
|  0%   29C    P8    23W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A40          Off  | 00000000:83:00.0 Off |                    0 |
|  0%   

In [2]:
# this one too!
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt 

2023-04-20 13:07:21.492288: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 13:07:23.956113: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/scratch/johannsm/python/lib:/usr/local/cuda-11.3/tensorRT-8.0/lib64:/usr/local/cuda-11.3/cudnn-8.2/lib64:/usr/local/cuda-11.3/nccl-2.9/lib64
2023-04-20 13:07:23.956252: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No su

In [16]:
# a note for part 1. you may use tf.random.categorical for sampling/running your markov chain.

A = tf.transpose(tf.constant([[0.3, 0.7], [0.5, 0.5]]))
A

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.3, 0.5],
       [0.7, 0.5]], dtype=float32)>

In [17]:
# let's sample from the first column
tf.random.categorical(A[:, 0], 1)  # second argument means 1 sample

InvalidArgumentError: {{function_node __wrapped__Multinomial_device_/job:localhost/replica:0/task:0/device:GPU:0}} logits should be a matrix, got shape [2] [Op:Multinomial]

In [26]:
# oh no! this function always needs a batch axis, so we add an axis in front
tf.random.categorical(A[tf.newaxis, :, 0], 1).numpy()

array([[1]])

In [27]:
# finally, the result includes axes both for the batch and number of samples,
# even though both are 1 here. so we can do:
tf.random.categorical(A[tf.newaxis, :, 0], 1)[0, 0].numpy()

# ...actually this is still wrong, because the function expects logits, not probabilities :)
# you figure it out!

1

In [28]:
# for part 2
import tensorflow_probability as tfp
tfd = tfp.distributions

In [31]:
# here is an example for a 2D mixture of Gaussians.
# we need a categorical distribution representing the mixture weights (here 0.5 for both components).
# as mixture components, we use a multivariate normal with diagonal covariance matrix.
# basically one component has mean [-location, -location], second component has mean [location location].
# both components use the same standard deviation (scale), although this could be changed.

location = 1.
scale = 1.
# "balanced" Gaussian mixture
basic_distribution = tfd.MixtureSameFamily(tfd.Categorical(probs=[0.5, 0.5]), 
                                           tfd.MultivariateNormalDiag(loc=[[-location, -location], [location, location]],
                                                                      scale_identity_multiplier=scale))

In [32]:
# simple way to sample from this distribution
basic_distribution.sample()

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-1.657833 , -0.6882014], dtype=float32)>

In [ ]:
# BUT NOW remember that we pretend that we can't actually sample from the above distribution.
# instead, we only sample conditional distributions p(x1 | x2) (or the other way around).

# since we sample one variable at a time, it's always a 1D Gaussian (tfd.Normal).
# the loc parameter looks a bit confusing: this is NOT a 2D Gaussian with mean (-location, location).
# instead, these are the locations for two SEPARATE 1D Gaussians, which are used for the mixture.
# both use the same scale.

# the main issue is figuring out the mixture components pi1, pi2.
# the derivation for this can be found in the stackexchange answer linked in the assignment text.
conditional_distribution = tfd.MixtureSameFamily(tfd.Categorical(probs=[pi1, pi2]),
                                                 tfd.Normal(loc=[-location, location], scale=scale))
conditional_distribution.sample()  # new x1

# NOTE as long as you are using independent Gaussians (diagonal covariance), the conditional distributions
# have the SAME mean/standard deviation as the 2D one has, in that dimension.
# the ONLY thing that changes are the mixture weights.

# for the example of sampling x1 given x2, they can be computed like this
mixing_score1 = prior_mixture_weight1 * tfd.Normal(loc=-location, scale=scale).prob(x2)
mixing_score2 = prior_mixture_weight2 * tfd.Normal(loc=location, scale=scale).prob(x2)
pi1 = mixing_score1 / (mixing_score1 + mixing_score2)
pi2 = mixing_score2 / (mixing_score1 + mixing_score2)

# for x2 given x1 you just flip it  -- use prob(x1) for the mixing scores.

In [ ]:
# in principle, this example could be extended to
# - unequal prior mixtures
# - more complex covariance structures
# - more than two components
# - non-Gaussian distributions
# - more than two dimensions (-> more Gibbs steps per sample)
# - ...